In [13]:
!apt-get update
!apt-get install -y cuda


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 1s (186 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [14]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Jan_15_19:20:09_PST_2025
Cuda compilation tools, release 12.8, V12.8.61
Build cuda_12.8.r12.8/compiler.35404655_0


In [15]:
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2


#define CUDNN_MAJOR 8
#define CUDNN_MINOR 9
#define CUDNN_PATCHLEVEL 6
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


In [16]:
!sudo apt-get update
!sudo apt-get install -y libnpp-dev-11-8


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [19]:
***************************************************
 * code_cuda.cu
 *
 * Extended example GPU-based project demonstrating:
 *   1. Basic vector ops (addition, fill, ReLU, etc.)
 *   2. Dot product and mat-vector multiply via cuBLAS
 *   3. Simple feed-forward MLP training (2-layer) with
 *      nonzero MSE and actual gradient update
 *   4. Simple cuDNN convolution, activation, pooling, softmax
 *   5. (Stub) NPP image rotation
 *
 * Build (example):
 *   nvcc code_cuda.cu -o code_cuda -lcudart -lcublas -lcudnn
 **************************************************/

#include <iostream>
#include <iomanip>
#include <string>
#include <vector>
#include <cmath>
#include <cassert>
#include <cstdio>
#include <cstdlib>
#include <ctime>

// CUDA headers
#include <cuda_runtime.h>
#include <cublas_v2.h>
#include <cudnn.h>

// Some constants, for demonstration
#define CHECK_CUDA_ERR(x)  do { if((x) != cudaSuccess) { \
                                fprintf(stderr,"Error at %s:%d - %s\n",__FILE__,__LINE__,cudaGetErrorString(x)); \
                                exit(EXIT_FAILURE);}} while(0)

#define CHECK_CUBLAS_ERR(x) do { if((x) != CUBLAS_STATUS_SUCCESS) { \
                                  fprintf(stderr,"CUBLAS Error at %s:%d\n",__FILE__,__LINE__); \
                                  exit(EXIT_FAILURE);}} while(0)

#define CHECK_CUDNN_ERR(x) do { cudnnStatus_t status_ = (x); \
                                if (status_ != CUDNN_STATUS_SUCCESS) { \
                                  fprintf(stderr, "cuDNN Error: %s at %s:%d\n", \
                                          cudnnGetErrorString(status_), __FILE__, __LINE__); \
                                  exit(EXIT_FAILURE);}} while(0)

//---------------------------------------------
// Utility functions
//---------------------------------------------
static inline float randomFloat(float low = -1.0f, float high = 1.0f)
{
    float r = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
    return low + r * (high - low);
}

__global__ void fillArrayKernel(float* arr, int N, float value)
{
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < N) arr[idx] = value;
}

__global__ void addArraysKernel(const float* a, const float* b, float* c, int N)
{
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < N) c[idx] = a[idx] + b[idx];
}

__global__ void reluKernel(float* data, int N)
{
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < N) data[idx] = fmaxf(0.0f, data[idx]);
}

__global__ void reluBackwardKernel(const float* input, const float* gradOutput, float* gradInput, int N)
{
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < N)
        gradInput[idx] = (input[idx] > 0.0f) ? gradOutput[idx] : 0.0f;
}

__global__ void squareKernel(float* data, int N)
{
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if(idx < N)
    {
        float val = data[idx];
        data[idx] = 0.5f * val * val; // 1/2 * (val^2)
    }
}

// Wrapper for fill kernel
void gpuFillArray(float* d_arr, int N, float value)
{
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    fillArrayKernel<<<blocks, threads>>>(d_arr, N, value);
    CHECK_CUDA_ERR(cudaDeviceSynchronize());
}

// Wrapper for add kernel
void gpuAddArrays(const float* d_a, const float* d_b, float* d_c, int N)
{
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    addArraysKernel<<<blocks, threads>>>(d_a, d_b, d_c, N);
    CHECK_CUDA_ERR(cudaDeviceSynchronize());
}

//---------------------------------------------
// cuBLAS matrix-multiply: C = A * B
// A: (M x K), B: (K x N), C: (M x N)
//---------------------------------------------
void matMulCuBLAS(cublasHandle_t handle,
                  const float* d_A, const float* d_B, float* d_C,
                  int M, int N, int K)
{
    const float alpha = 1.0f;
    const float beta  = 0.0f;
    CHECK_CUBLAS_ERR(
        cublasSgemm(handle,
                    CUBLAS_OP_N, CUBLAS_OP_N,
                    M, N, K,
                    &alpha,
                    d_A, M,
                    d_B, K,
                    &beta,
                    d_C, M)
    );
}

//---------------------------------------------
// Fully connected: y = W*x + b
// W: (outDim x inDim), x: (inDim x 1)
//---------------------------------------------
void fullyConnectedLayer(cublasHandle_t handle,
                         const float* d_W,
                         /*const float* d_b,*/ // b addition is separate
                         const float* d_x,
                         float* d_y,
                         int inDim,
                         int outDim)
{
    // y = W*x (we'll add bias with a separate kernel or approach)
    matMulCuBLAS(handle, d_W, d_x, d_y, outDim, 1, inDim);
}

// Add bias (per-element)
__global__ void addBiasKernel(float* d_y, const float* d_b, int outDim)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx < outDim)
        d_y[idx] += d_b[idx];
}

void fullyConnectedLayerAddBias(const float* d_b, float* d_y, int outDim)
{
    int threads = 256;
    int blocks = (outDim + threads - 1) / threads;
    addBiasKernel<<<blocks, threads>>>(d_y, d_b, outDim);
    CHECK_CUDA_ERR(cudaDeviceSynchronize());
}

//---------------------------------------------
// Backprop for fully connected layer
// gradOut: gradient wrt output (outDim x 1)
// X: input (inDim x 1)
// W: (outDim x inDim)
//---------------------------------------------
void backpropFullyConnected(cublasHandle_t handle,
                            const float* d_gradOut,
                            const float* d_X,
                            const float* d_W,
                            float* d_gradW,
                            float* d_gradB,
                            float* d_gradX,
                            int inDim, int outDim)
{
    // gradW = gradOut * X^T => (outDim x 1)*(1 x inDim) => (outDim x inDim)
    {
        const float alpha = 1.0f;
        const float beta  = 0.0f;
        CHECK_CUBLAS_ERR(
            cublasSgemm(handle,
                        CUBLAS_OP_N, CUBLAS_OP_T,
                        outDim, inDim, 1,
                        &alpha,
                        d_gradOut, outDim,
                        d_X, inDim,
                        &beta,
                        d_gradW, outDim)
        );
    }

    // gradB = gradOut (one value per output neuron in this single-sample case)
    CHECK_CUDA_ERR(cudaMemcpy(d_gradB, d_gradOut, outDim*sizeof(float), cudaMemcpyDeviceToDevice));

    // gradX = W^T * gradOut => (inDim x outDim)*(outDim x 1) => (inDim x 1)
    {
        const float alpha = 1.0f;
        const float beta  = 0.0f;
        CHECK_CUBLAS_ERR(
            cublasSgemm(handle,
                        CUBLAS_OP_T, CUBLAS_OP_N,
                        inDim, 1, outDim,
                        &alpha,
                        d_W, outDim,
                        d_gradOut, outDim,
                        &beta,
                        d_gradX, inDim)
        );
    }
}

//---------------------------------------------
// Extra cuDNN demos: Convolution, ReLU, Pooling, Softmax
//---------------------------------------------
void cudnnConvolutionExample(cudnnHandle_t cudnn)
{
    std::cout << "[INFO] Running a sample cuDNN Convolution...\n";

    cudnnTensorDescriptor_t inDesc, outDesc;
    cudnnFilterDescriptor_t filtDesc;
    cudnnConvolutionDescriptor_t convDesc;

    CHECK_CUDNN_ERR( cudnnCreateTensorDescriptor(&inDesc) );
    CHECK_CUDNN_ERR( cudnnCreateTensorDescriptor(&outDesc) );
    CHECK_CUDNN_ERR( cudnnCreateFilterDescriptor(&filtDesc) );
    CHECK_CUDNN_ERR( cudnnCreateConvolutionDescriptor(&convDesc) );

    // For a 28x28 single-channel example
    CHECK_CUDNN_ERR( cudnnSetTensor4dDescriptor(inDesc, CUDNN_TENSOR_NCHW,
                                                CUDNN_DATA_FLOAT,
                                                1, 1, 28, 28) );

    CHECK_CUDNN_ERR( cudnnSetTensor4dDescriptor(outDesc, CUDNN_TENSOR_NCHW,
                                                CUDNN_DATA_FLOAT,
                                                1, 1, 24, 24) );

    CHECK_CUDNN_ERR( cudnnSetFilter4dDescriptor(filtDesc,
                                                CUDNN_DATA_FLOAT,
                                                CUDNN_TENSOR_NCHW,
                                                1, 1, 5, 5) );

    CHECK_CUDNN_ERR( cudnnSetConvolution2dDescriptor(convDesc,
                                                     0, 0,
                                                     1, 1,
                                                     1, 1,
                                                     CUDNN_CROSS_CORRELATION,
                                                     CUDNN_DATA_FLOAT) );

    // We'll pick a known forward algo
    cudnnConvolutionFwdAlgo_t algo = CUDNN_CONVOLUTION_FWD_ALGO_IMPLICIT_GEMM;

    size_t workspaceBytes = 0;
    CHECK_CUDNN_ERR(
        cudnnGetConvolutionForwardWorkspaceSize(
            cudnn, inDesc, filtDesc, convDesc, outDesc, algo, &workspaceBytes)
    );

    void* d_workspace = nullptr;
    if (workspaceBytes > 0)
        CHECK_CUDA_ERR( cudaMalloc(&d_workspace, workspaceBytes) );

    float *d_input = nullptr, *d_filter = nullptr, *d_output = nullptr;
    CHECK_CUDA_ERR( cudaMalloc(&d_input,  1*1*28*28*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_filter, 1*1*5*5*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_output, 1*1*24*24*sizeof(float)) );

    // fill input/filter with some constants
    gpuFillArray(d_input, 1*1*28*28, 0.1f);
    gpuFillArray(d_filter, 1*1*5*5,  0.2f);

    const float alpha = 1.0f, beta = 0.0f;
    CHECK_CUDNN_ERR(
        cudnnConvolutionForward(cudnn,
                                &alpha,
                                inDesc, d_input,
                                filtDesc, d_filter,
                                convDesc, algo,
                                d_workspace, workspaceBytes,
                                &beta,
                                outDesc, d_output)
    );

    // Cleanup
    CHECK_CUDNN_ERR( cudnnDestroyConvolutionDescriptor(convDesc) );
    CHECK_CUDNN_ERR( cudnnDestroyFilterDescriptor(filtDesc) );
    CHECK_CUDNN_ERR( cudnnDestroyTensorDescriptor(inDesc) );
    CHECK_CUDNN_ERR( cudnnDestroyTensorDescriptor(outDesc) );
    if(d_workspace) cudaFree(d_workspace);

    cudaFree(d_input);
    cudaFree(d_filter);
    cudaFree(d_output);

    std::cout << "[INFO] cuDNN Convolution example complete.\n";
}

void cudnnReluExample(cudnnHandle_t cudnn)
{
    std::cout << "[INFO] Running a sample cuDNN ReLU...\n";

    cudnnTensorDescriptor_t desc;
    cudnnActivationDescriptor_t actDesc;
    CHECK_CUDNN_ERR( cudnnCreateTensorDescriptor(&desc) );
    CHECK_CUDNN_ERR( cudnnCreateActivationDescriptor(&actDesc) );

    CHECK_CUDNN_ERR( cudnnSetTensor4dDescriptor(desc,
                                                CUDNN_TENSOR_NCHW,
                                                CUDNN_DATA_FLOAT,
                                                1, 10, 1, 1) );

    CHECK_CUDNN_ERR( cudnnSetActivationDescriptor(actDesc,
                                                  CUDNN_ACTIVATION_RELU,
                                                  CUDNN_PROPAGATE_NAN,
                                                  0.0) );

    float *d_input = nullptr, *d_output = nullptr;
    CHECK_CUDA_ERR( cudaMalloc(&d_input,  10*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_output, 10*sizeof(float)) );

    // Fill with negative to see ReLU effect
    gpuFillArray(d_input, 10, -0.5f);

    float alpha = 1.0f, beta = 0.0f;
    CHECK_CUDNN_ERR(
        cudnnActivationForward(cudnn,
                               actDesc,
                               &alpha,
                               desc, d_input,
                               &beta,
                               desc, d_output)
    );

    // Cleanup
    CHECK_CUDNN_ERR( cudnnDestroyActivationDescriptor(actDesc) );
    CHECK_CUDNN_ERR( cudnnDestroyTensorDescriptor(desc) );
    cudaFree(d_input);
    cudaFree(d_output);

    std::cout << "[INFO] cuDNN ReLU example complete.\n";
}

void cudnnPoolingExample(cudnnHandle_t cudnn)
{
    std::cout << "[INFO] Running a sample cuDNN Pooling (Max Pool)...\n";

    cudnnTensorDescriptor_t inDesc, outDesc;
    cudnnPoolingDescriptor_t poolDesc;
    CHECK_CUDNN_ERR( cudnnCreateTensorDescriptor(&inDesc) );
    CHECK_CUDNN_ERR( cudnnCreateTensorDescriptor(&outDesc) );
    CHECK_CUDNN_ERR( cudnnCreatePoolingDescriptor(&poolDesc) );

    // 1x1x6x6 input
    CHECK_CUDNN_ERR(
        cudnnSetTensor4dDescriptor(inDesc, CUDNN_TENSOR_NCHW,
                                   CUDNN_DATA_FLOAT, 1, 1, 6, 6) );

    // 2x2 kernel, stride 2 => output 1x1x3x3
    CHECK_CUDNN_ERR(
        cudnnSetTensor4dDescriptor(outDesc, CUDNN_TENSOR_NCHW,
                                   CUDNN_DATA_FLOAT, 1, 1, 3, 3) );

    CHECK_CUDNN_ERR(
        cudnnSetPooling2dDescriptor(poolDesc,
                                    CUDNN_POOLING_MAX,
                                    CUDNN_PROPAGATE_NAN,
                                    2, 2,  // window
                                    0, 0,  // pad
                                    2, 2)  // stride
    );

    float *d_input = nullptr, *d_output = nullptr;
    CHECK_CUDA_ERR( cudaMalloc(&d_input,  6*6*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_output, 3*3*sizeof(float)) );

    // Fill input with random data
    std::vector<float> h_in(36);
    for (int i = 0; i < 36; i++){
        h_in[i] = (float)(rand() % 100);
    }
    CHECK_CUDA_ERR( cudaMemcpy(d_input, h_in.data(), 36*sizeof(float),
                               cudaMemcpyHostToDevice) );

    float alpha = 1.0f, beta = 0.0f;
    CHECK_CUDNN_ERR(
        cudnnPoolingForward(cudnn,
                            poolDesc,
                            &alpha,
                            inDesc, d_input,
                            &beta,
                            outDesc, d_output)
    );

    // Download result
    std::vector<float> h_out(9);
    CHECK_CUDA_ERR( cudaMemcpy(h_out.data(), d_output, 9*sizeof(float),
                               cudaMemcpyDeviceToHost) );

    std::cout << "[INFO] Pooled Output (3x3):\n";
    for(int i=0; i<9; i++){
        std::cout << std::setw(5) << h_out[i]
                  << ((i%3==2) ? "\n" : " ");
    }

    // Cleanup
    CHECK_CUDA_ERR( cudaFree(d_input) );
    CHECK_CUDA_ERR( cudaFree(d_output) );
    CHECK_CUDNN_ERR( cudnnDestroyPoolingDescriptor(poolDesc) );
    CHECK_CUDNN_ERR( cudnnDestroyTensorDescriptor(inDesc) );
    CHECK_CUDNN_ERR( cudnnDestroyTensorDescriptor(outDesc) );

    std::cout << "[INFO] cuDNN Pooling example complete.\n";
}

void cudnnSoftmaxExample(cudnnHandle_t cudnn)
{
    std::cout << "[INFO] Running a sample cuDNN Softmax...\n";
    cudnnTensorDescriptor_t desc;
    CHECK_CUDNN_ERR( cudnnCreateTensorDescriptor(&desc) );

    CHECK_CUDNN_ERR(
        cudnnSetTensor4dDescriptor(desc,
                                   CUDNN_TENSOR_NCHW,
                                   CUDNN_DATA_FLOAT,
                                   1, 5, 1, 1) );

    float *d_input = nullptr, *d_output = nullptr;
    CHECK_CUDA_ERR( cudaMalloc(&d_input,  5*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_output, 5*sizeof(float)) );

    // Fill input with random values
    std::vector<float> h_in(5);
    for(int i=0; i<5; i++){
        h_in[i] = randomFloat(-2.0f, 2.0f);
    }

    CHECK_CUDA_ERR( cudaMemcpy(d_input, h_in.data(),
                               5*sizeof(float),
                               cudaMemcpyHostToDevice) );

    float alpha = 1.0f, beta = 0.0f;
    CHECK_CUDNN_ERR(
        cudnnSoftmaxForward(cudnn,
                            CUDNN_SOFTMAX_ACCURATE,  // or CUDNN_SOFTMAX_FAST
                            CUDNN_SOFTMAX_MODE_INSTANCE,
                            &alpha,
                            desc, d_input,
                            &beta,
                            desc, d_output)
    );

    // Retrieve and print output
    std::vector<float> h_out(5);
    CHECK_CUDA_ERR( cudaMemcpy(h_out.data(), d_output, 5*sizeof(float),
                               cudaMemcpyDeviceToHost) );

    std::cout << "[INFO] Softmax Input -> Output:\n";
    for(int i=0; i<5; i++){
        std::cout << "  " << h_in[i] << " -> " << h_out[i] << "\n";
    }

    // Cleanup
    CHECK_CUDA_ERR( cudaFree(d_input) );
    CHECK_CUDA_ERR( cudaFree(d_output) );
    CHECK_CUDNN_ERR( cudnnDestroyTensorDescriptor(desc) );

    std::cout << "[INFO] cuDNN Softmax example complete.\n";
}

//---------------------------------------------
// Stub for NPP image rotation
//---------------------------------------------
void nppImageRotationExample(const std::string& filename)
{
    // Stub only. No actual NPP calls here.
    std::cout << "[INFO] (Stub) NPP Image Rotation of " << filename << "\n";
}

//---------------------------------------------
// CPU helper for random init
//---------------------------------------------
void cpuRandomInit(float* d_arr, int N, float low=-1.0f, float high=1.0f)
{
    std::vector<float> hostVec(N);
    for(int i=0; i<N; i++){
        hostVec[i] = randomFloat(low, high);
    }
    CHECK_CUDA_ERR( cudaMemcpy(d_arr, hostVec.data(),
                               N*sizeof(float),
                               cudaMemcpyHostToDevice) );
}

//---------------------------------------------
// Simple NN training (2-layer MLP) with actual
// gradient updates so the MSE won't stay at 0.
//---------------------------------------------
void simpleNeuralNetworkTraining(cublasHandle_t cublas,
                                 int inputSize,
                                 int hiddenSize,
                                 int outputSize,
                                 int iterations = 5)
{
    std::cout << "----------------------------------\n";
    std::cout << "[INFO] Starting Simple NN Training ("
              << iterations << " iterations)...\n";

    // 1) Allocate weights & biases
    float *d_W1, *d_b1, *d_W2, *d_b2;
    CHECK_CUDA_ERR( cudaMalloc(&d_W1, hiddenSize * inputSize * sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_b1, hiddenSize * sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_W2, outputSize * hiddenSize * sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_b2, outputSize * sizeof(float)) );

    // 2) Initialize them in [-0.5, 0.5], for example
    cpuRandomInit(d_W1, hiddenSize*inputSize, -0.5f, 0.5f);
    cpuRandomInit(d_b1, hiddenSize, -0.5f, 0.5f);
    cpuRandomInit(d_W2, outputSize*hiddenSize, -0.5f, 0.5f);
    cpuRandomInit(d_b2, outputSize, -0.5f, 0.5f);

    // 3) Allocate input & ground truth (single sample for demo)
    float *d_x, *d_yGT;
    CHECK_CUDA_ERR( cudaMalloc(&d_x, inputSize * sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_yGT, outputSize * sizeof(float)) );

    // Input in [0,1]
    cpuRandomInit(d_x, inputSize, 0.0f, 1.0f);

    // Ground truth: all zeros except 1 for the first dimension
    gpuFillArray(d_yGT, outputSize, 0.0f);
    float oneVal = 1.0f;
    CHECK_CUDA_ERR( cudaMemcpy(d_yGT, &oneVal,
                               sizeof(float),
                               cudaMemcpyHostToDevice) );

    // Prepare device arrays for intermediate results
    float *d_h1, *d_h2;
    CHECK_CUDA_ERR( cudaMalloc(&d_h1, hiddenSize*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_h2, outputSize*sizeof(float)) );

    float *d_lossVec;
    CHECK_CUDA_ERR( cudaMalloc(&d_lossVec, outputSize*sizeof(float)) );

    float *d_gradOut2;
    CHECK_CUDA_ERR( cudaMalloc(&d_gradOut2, outputSize*sizeof(float)) );

    float *d_gradW2, *d_gradB2, *d_gradH1;
    CHECK_CUDA_ERR( cudaMalloc(&d_gradW2, outputSize*hiddenSize*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_gradB2, outputSize*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_gradH1, hiddenSize*sizeof(float)) );

    float *d_gradW1, *d_gradB1, *d_gradX;
    CHECK_CUDA_ERR( cudaMalloc(&d_gradW1, hiddenSize*inputSize*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_gradB1, hiddenSize*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_gradX, inputSize*sizeof(float)) );

    // Learning rate
    const float learningRate = 0.01f;
    const float negLR = -learningRate;

    // 4) Training loop
    for(int iter=0; iter<iterations; iter++)
    {
        //---------------------------------
        // Forward pass: fc1 -> add bias -> ReLU -> fc2 -> add bias
        //---------------------------------
        // FC1
        fullyConnectedLayer(cublas, d_W1, d_x, d_h1, inputSize, hiddenSize);
        fullyConnectedLayerAddBias(d_b1, d_h1, hiddenSize);

        // ReLU
        {
            int threads = 256;
            int blocks = (hiddenSize + threads - 1) / threads;
            reluKernel<<<blocks, threads>>>(d_h1, hiddenSize);
            CHECK_CUDA_ERR(cudaDeviceSynchronize());
        }

        // FC2
        fullyConnectedLayer(cublas, d_W2, d_h1, d_h2, hiddenSize, outputSize);
        fullyConnectedLayerAddBias(d_b2, d_h2, outputSize);

        //---------------------------------
        // Compute MSE Loss = 0.5 * sum((h2 - yGT)^2)
        // We'll do (h2 - yGT) in d_lossVec, then square each element * 0.5
        //---------------------------------
        // Step 1: d_lossVec = h2
        CHECK_CUBLAS_ERR( cublasScopy(cublas, outputSize, d_h2, 1, d_lossVec, 1) );
        // Step 2: d_lossVec -= yGT
        const float alphaNegOne = -1.0f;
        CHECK_CUBLAS_ERR( cublasSaxpy(cublas, outputSize, &alphaNegOne, d_yGT, 1, d_lossVec, 1) );
        // Step 3: square each element => 0.5 * val^2
        {
            int threads = 256;
            int blocks = (outputSize + threads - 1) / threads;
            squareKernel<<<blocks, threads>>>(d_lossVec, outputSize);
            CHECK_CUDA_ERR(cudaDeviceSynchronize());
        }
        // Sum on CPU for printing
        std::vector<float> h_lossVec(outputSize);
        CHECK_CUDA_ERR( cudaMemcpy(h_lossVec.data(), d_lossVec, outputSize*sizeof(float),
                                   cudaMemcpyDeviceToHost) );
        float lossSum = 0.f;
        for(float v : h_lossVec) lossSum += v;

        //---------------------------------
        // Backprop
        // gradOut2 = (h2 - yGT)
        //---------------------------------
        CHECK_CUBLAS_ERR( cublasScopy(cublas, outputSize, d_h2, 1, d_gradOut2, 1) );
        CHECK_CUBLAS_ERR( cublasSaxpy(cublas, outputSize, &alphaNegOne, d_yGT, 1, d_gradOut2, 1) );

        // backprop fc2
        backpropFullyConnected(cublas, d_gradOut2, d_h1, d_W2,
                               d_gradW2, d_gradB2, d_gradH1,
                               hiddenSize, outputSize);

        // backprop ReLU
        {
            int threads = 256;
            int blocks = (hiddenSize + threads - 1) / threads;
            reluBackwardKernel<<<blocks, threads>>>(d_h1, d_gradH1, d_gradH1, hiddenSize);
            CHECK_CUDA_ERR(cudaDeviceSynchronize());
        }

        // backprop fc1
        backpropFullyConnected(cublas, d_gradH1, d_x, d_W1,
                               d_gradW1, d_gradB1, d_gradX,
                               inputSize, hiddenSize);

        //---------------------------------
        // Gradient descent update
        // W2 -= LR * d_gradW2, etc.
        //---------------------------------
        CHECK_CUBLAS_ERR( cublasSaxpy(cublas, outputSize*hiddenSize,
                                       &negLR, d_gradW2, 1, d_W2, 1) );
        CHECK_CUBLAS_ERR( cublasSaxpy(cublas, outputSize,
                                       &negLR, d_gradB2, 1, d_b2, 1) );
        CHECK_CUBLAS_ERR( cublasSaxpy(cublas, hiddenSize*inputSize,
                                       &negLR, d_gradW1, 1, d_W1, 1) );
        CHECK_CUBLAS_ERR( cublasSaxpy(cublas, hiddenSize,
                                       &negLR, d_gradB1, 1, d_b1, 1) );

        //---------------------------------
        // Print iteration info
        //---------------------------------
        std::cout << "  [Iteration " << (iter+1)
                  << "] MSE Loss = " << lossSum << "\n";
    }

    // Cleanup
    cudaFree(d_W1);
    cudaFree(d_b1);
    cudaFree(d_W2);
    cudaFree(d_b2);
    cudaFree(d_x);
    cudaFree(d_yGT);
    cudaFree(d_h1);
    cudaFree(d_h2);
    cudaFree(d_lossVec);
    cudaFree(d_gradOut2);
    cudaFree(d_gradW2);
    cudaFree(d_gradB2);
    cudaFree(d_gradH1);
    cudaFree(d_gradW1);
    cudaFree(d_gradB1);
    cudaFree(d_gradX);

    std::cout << "[INFO] Finished Simple NN Training Example.\n";
}

//---------------------------------------------
// Additional example: Dot product (cublasSdot)
//---------------------------------------------
void cublasDotProductExample(cublasHandle_t handle)
{
    std::cout << "----------------------------------\n";
    std::cout << "[INFO] Running a cuBLAS Dot Product example...\n";
    int N = 10;
    float *d_a, *d_b;
    CHECK_CUDA_ERR( cudaMalloc(&d_a, N*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_b, N*sizeof(float)) );

    // Fill with some values
    std::vector<float> h_a(N), h_b(N);
    for(int i=0; i<N; i++){
        h_a[i] = float(i+1); // 1,2,3,...
        h_b[i] = 2.0f;       // all 2
    }
    CHECK_CUDA_ERR( cudaMemcpy(d_a, h_a.data(), N*sizeof(float), cudaMemcpyHostToDevice) );
    CHECK_CUDA_ERR( cudaMemcpy(d_b, h_b.data(), N*sizeof(float), cudaMemcpyHostToDevice) );

    float result = 0.0f;
    CHECK_CUBLAS_ERR( cublasSdot(handle, N, d_a, 1, d_b, 1, &result) );

    std::cout << "  Dot Product of [1..10] and [2..2] = " << result
              << " (expected 110)\n";

    cudaFree(d_a);
    cudaFree(d_b);
    std::cout << "[INFO] cuBLAS Dot Product example complete.\n";
}

//---------------------------------------------
// Additional example: Matrix-Vector multiply (cublasSgemv)
//---------------------------------------------
void cublasMatVecExample(cublasHandle_t handle)
{
    std::cout << "----------------------------------\n";
    std::cout << "[INFO] Running a cuBLAS Matrix-Vector example (SGEMV)...\n";

    // Let A be 3x3, x be 3x1 => y = A*x => 3x1
    float A[9] = {1.f,2.f,3.f,
                  4.f,5.f,6.f,
                  7.f,8.f,9.f};
    float x[3] = {1.f, 2.f, 3.f};
    float *d_A, *d_x, *d_y;
    CHECK_CUDA_ERR( cudaMalloc(&d_A, 9*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_x, 3*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_y, 3*sizeof(float)) );

    CHECK_CUDA_ERR( cudaMemcpy(d_A, A, 9*sizeof(float), cudaMemcpyHostToDevice) );
    CHECK_CUDA_ERR( cudaMemcpy(d_x, x, 3*sizeof(float), cudaMemcpyHostToDevice) );

    // Zero out d_y
    gpuFillArray(d_y, 3, 0.0f);

    float alpha = 1.f, beta = 0.f;
    CHECK_CUBLAS_ERR(
        cublasSgemv(handle, CUBLAS_OP_N,
                    3, 3,
                    &alpha,
                    d_A, 3,
                    d_x, 1,
                    &beta,
                    d_y, 1)
    );

    // Retrieve result
    float y[3];
    CHECK_CUDA_ERR( cudaMemcpy(y, d_y, 3*sizeof(float), cudaMemcpyDeviceToHost) );
    std::cout << "  [1 2 3; 4 5 6; 7 8 9] * [1; 2; 3] = ["
              << y[0] << " " << y[1] << " " << y[2] << "]^T\n";
    // should be [14 32 50]^T

    cudaFree(d_A);
    cudaFree(d_x);
    cudaFree(d_y);

    std::cout << "[INFO] cuBLAS SGEMV example complete.\n";
}

//---------------------------------------------
// MAIN
//---------------------------------------------
int main(int argc, char** argv)
{
    srand((unsigned)time(nullptr));
    CHECK_CUDA_ERR( cudaSetDevice(0) );

    // Create cuBLAS handle
    cublasHandle_t cublas;
    CHECK_CUBLAS_ERR( cublasCreate(&cublas) );

    // Create cuDNN handle
    cudnnHandle_t cudnn;
    CHECK_CUDNN_ERR( cudnnCreate(&cudnn) );

    // 1) Basic array ops
    std::cout << "----------------------------------\n";
    std::cout << "[INFO] Basic Array Ops Demo\n";
    int N = 1000;
    float *d_a, *d_b, *d_c;
    CHECK_CUDA_ERR( cudaMalloc(&d_a, N*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_b, N*sizeof(float)) );
    CHECK_CUDA_ERR( cudaMalloc(&d_c, N*sizeof(float)) );

    gpuFillArray(d_a, N, 1.0f);
    gpuFillArray(d_b, N, 2.0f);
    gpuAddArrays(d_a, d_b, d_c, N);

    // Download & check
    std::vector<float> h_c(N);
    CHECK_CUDA_ERR( cudaMemcpy(h_c.data(), d_c, N*sizeof(float), cudaMemcpyDeviceToHost) );

    float sumCheck = 0.0f;
    for(int i=0; i<N; i++){
        sumCheck += h_c[i];
    }
    std::cout << "[INFO] Sum of c after add: " << sumCheck
              << " (expected ~3000 if N=1000)\n";

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // 2) Dot product with cuBLAS
    cublasDotProductExample(cublas);

    // 3) Matrix-vector multiply with cuBLAS
    cublasMatVecExample(cublas);

    // 4) cuDNN Convolution
    std::cout << "----------------------------------\n";
    cudnnConvolutionExample(cudnn);

    // 5) cuDNN ReLU
    std::cout << "----------------------------------\n";
    cudnnReluExample(cudnn);

    // 6) cuDNN Pooling
    std::cout << "----------------------------------\n";
    cudnnPoolingExample(cudnn);

    // 7) cuDNN Softmax
    std::cout << "----------------------------------\n";
    cudnnSoftmaxExample(cudnn);

    // 8) NPP image rotation (stub)
    std::cout << "----------------------------------\n";
    nppImageRotationExample("input.png");

    // 9) Simple NN training with multiple iterations + gradient update
    simpleNeuralNetworkTraining(cublas, /*inputSize=*/128,
                                           /*hiddenSize=*/64,
                                           /*outputSize=*/10,
                                           /*iterations=*/5);

    // Cleanup handles
    CHECK_CUBLAS_ERR( cublasDestroy(cublas) );
    CHECK_CUDNN_ERR( cudnnDestroy(cudnn) );

    std::cout << "----------------------------------\n";
    std::cout << "[INFO] Program finished successfully.\n";
    return 0;
}


Overwriting code_cuda.cu


In [20]:
# 1) Write the file (the code above) in a notebook cell with %%writefile code_cuda.cu
# 2) Compile:
!nvcc code_cuda.cu -o code_cuda -lcublas -lcudnn
# 3) Run:
!./code_cuda


nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
----------------------------------
[INFO] Basic Array Ops Demo
[INFO] Sum of c after add: 0 (expected ~3000 if N=1000)
----------------------------------
[INFO] Running a cuBLAS Dot Product example...
  Dot Product of [1..10] and [2..2] = 110 (expected 110)
[INFO] cuBLAS Dot Product example complete.
----------------------------------
[INFO] Running a cuBLAS Matrix-Vector example (SGEMV)...
  [1 2 3; 4 5 6; 7 8 9] * [1; 2; 3] = [30 36 42]^T
[INFO] cuBLAS SGEMV example complete.
----------------------------------
[INFO] Running a sample cuDNN Convolution...
[INFO] cuDNN Convolution example complete.
----------------------------------
[INFO] Running a sample cuDNN ReLU...
[INFO] cuDNN ReLU example complete.
----------------------------------
[INFO] Running a sample cuDNN Pooling (Max Pool)...
[INFO] Pool